# **Imports**

In [1]:
import os
os.chdir(os.path.expanduser("~/PycharmProjects/vivarium-ecoli"))

In [ ]:
import argparse
import numpy as np
import pathlib, datetime

# vivarium-core imports
import pytest
from vivarium.core.engine import Engine
from vivarium.core.composer import Composer
from vivarium.library.dict_utils import deep_merge

# vivarium-ecoli imports
from ecoli.experiments.ecoli_master_sim import EcoliSim, CONFIG_DIR_PATH
from ecoli.library.sim_data import LoadSimData
from ecoli.states.wcecoli_state import get_state_from_file
from ecoli.composites.ecoli_master import SIM_DATA_PATH
from ecoli.processes.registries import topology_registry

In [ ]:
# **Parameters**

In [ ]:
filename='default'
total_time=10
divide=True
initial_state_file='vivecoli_t2'
progress_bar=True
log_updates=False
emitter='timeseries'

In [ ]:
sim = EcoliSim.from_file(CONFIG_DIR_PATH + filename + '.json')
sim.total_time = total_time
sim.raw_output = False
sim.divide = divide
sim.progress_bar = progress_bar
sim.log_updates = log_updates
sim.emitter = emitter
sim.initial_state = get_state_from_file(path=f'data/{initial_state_file}.json')

In [ ]:
sim.build_ecoli()

In [ ]:
sim.processes

# **Run Simulation**

In [ ]:
sim.run()

# **Query Output**

In [ ]:
sim.query([ ('agents', 0, 'listeners', 'mass')])

# **Query Output**

In [12]:
sim.query([ ('agents', 0, 'listeners', 'mass')])

{'agents': {0: {'listeners': {'mass': [None, None, None, None, None, None]}}},
 'time': [0.0, 2.0, 4.0, 6.0, 8.0, 10.0]}

In [8]:
folderName = input("Enter the output Folder Path. If prefer default, press ENTER:")

if folderName == "":
    folder = f'out/geneRxnVerifData'
else:
    folder = f'{folderName}'

Enter the output Folder Path. If prefer default, press ENTER: 


# *Enter Query Parameters*

In [ ]:
query = []
agents = sim.query()['agents'].keys()
for agent in agents:
    query.extend([('agents', agent, 'listeners', 'fba_results'),
                  ('agents', agent, 'bulk'), ('agents', agent, 'listeners', 'mass')])
output = sim.query(query)

pathlib.Path(folder).mkdir(parents=True, exist_ok=True)